In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
df_train = pd.read_csv('./data/train.csv').drop_duplicates()
df_test = pd.read_csv('./data/test.csv').drop_duplicates()
df_test

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import KFold, cross_val_score, cross_val_predict, GridSearchCV
from sklearn.feature_selection import RFE
def linearRegression(X_train, Y_train, X_test, Y_test, i):
    folds = KFold(n_splits = 5, shuffle = True, random_state = 100)
    hyper_params = [{'n_features_to_select': list(range(1, 20))}]

    model = LinearRegression()
    # model.fit(X_train, Y_train)
    print(i)
    rfe = RFE(model)

    
    model_cv = GridSearchCV(estimator=rfe, param_grid=hyper_params, scoring='r2', cv = 5, return_train_score=True, verbose = 1)    
    predicted = model_cv.fit(X_train, Y_train).predict(X_test)
    # predicted = cross_val_predict(model, X_test, Y_test, cv = folds)

    return predicted

In [ ]:
from sklearn.metrics import precision_score
X_train = df_train.drop(columns = ['results_positionOrder'])
Y_train = df_train['results_positionOrder']

score = 0
for i in df_test['raceId'].unique():
    X_test = df_test[df_test['raceId'] == i].drop(columns = ['results_positionOrder'])
    Y_test = df_test[df_test['raceId'] == i]['results_positionOrder']

    prediction_df = pd.DataFrame(linearRegression(X_train, Y_train, X_test, Y_test, i), columns = ['results'])
    prediction_df['podium'] = Y_test.reset_index(drop = True)
    prediction_df['actual'] = prediction_df.podium.map(lambda x: 1 if x == 1 else 0)
    prediction_df.sort_values('results', ascending = True, inplace = True)
    prediction_df.reset_index(inplace = True, drop = True)
    prediction_df['predicted'] = prediction_df.index
    prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

    score += precision_score(prediction_df.actual, prediction_df.predicted)
modelScore = score / df_test['raceId'].nunique()
modelScore
